# Step 1 - Frame Dividing Part

#### Install Opencv for Frame Extraction

In [ ]:
pip install opencv-python-headless

#### Importing Libraries CV2 and OS

In [ ]:
import cv2
import os

#### Function to Extract Frames

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=30):
    """Extract frames from a video at a given frame rate."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get video FPS

    # Debug: Print FPS
    print(f"Video: {video_path}, FPS: {fps}")

    if fps == 0:
        print("Error: Could not read FPS. Check the video file or codec.")
        return  # Exit function to prevent division by zero

    frame_interval = max(1, int(fps / frame_rate))  # Avoid division by zero

    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop when the video ends

        if count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_id:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_id += 1  # Increment frame number

        count += 1

    cap.release()
    print(f" Extracted {frame_id} frames from {video_path}")


##### Extracting frames

In [ ]:
extract_frames("Video/Player02_FS_Set01_F_50Kg.MOV", "Output Video/frames/front", frame_rate=30)
extract_frames("Video/Player02_FS_Set01_S_50Kg.MOV", "Output Video/frames/side", frame_rate=30)
extract_frames("Video/Player02_FS_Set01_A_50Kg.mp4", "Output Video/frames/top", frame_rate=30)


#### Check the orientation and Quality

In [ ]:
import cv2
import os

def check_orientation(folder_path):
    """Check the orientation of images in a folder."""
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if image is not loaded

        height, width, _ = img.shape
        orientation = "Landscape" if width > height else "Portrait"
        print(f"{img_name}: {orientation} ({width}x{height})")

# Example Usage
check_orientation("Output Video/frames/front")
check_orientation("Output Video/frames/side")
check_orientation("Output Video/frames/top")


#### Rotate frames if needed

In [ ]:
def rotate_images(folder_path, rotate_code):
    """Rotate all images in a folder."""
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  # Skip corrupted files

        rotated = cv2.rotate(img, rotate_code)
        cv2.imwrite(img_path, rotated)
        print(f"Rotated {img_name}")

# Rotate 90 degrees clockwise
rotate_images("Output Video/frames/front", cv2.ROTATE_90_CLOCKWISE)

# Rotate 90 degrees counter-clockwise
# rotate_images("Output Video/frames/side", cv2.ROTATE_90_COUNTERCLOCKWISE)


#### Remove Blurry Images from frames

In [ ]:
def analyze_blurriness(folder_path):
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        print(f"{img_name}: Variance = {laplacian_var:.2f}")


In [ ]:
analyze_blurriness("Output Video/frames/top")

In [ ]:
import shutil

def move_blurry_images(folder_path, threshold=100.0, blurry_folder="blurry_top"):
    os.makedirs(blurry_folder, exist_ok=True)
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

        if laplacian_var < threshold:
            shutil.move(img_path, os.path.join(blurry_folder, img_name))
            print(f"Moved blurry image: {img_name} (Variance = {laplacian_var:.2f})")


# Step 2 - Extracting Key points

#### Install mediapipe

In [ ]:
pip install mediapipe

#### Detect Keypoints - Annotate the Frames

In [ ]:
import cv2
import mediapipe as mp
import os

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def detect_keypoints(input_folder, output_folder):
    """Detects keypoints in images and saves annotated frames."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in sorted(os.listdir(input_folder)):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue  # Skip if image is corrupted

        # Convert BGR to RGB (MediaPipe expects RGB)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_image)

        if results.pose_landmarks:
            # Draw keypoints on the image
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Save the output image
        output_path = os.path.join(output_folder, img_name)
        cv2.imwrite(output_path, image)

    print(f"✅ Keypoint detection completed. Output saved in {output_folder}")

# Example usage
detect_keypoints("Output Video/frames/front", "Output Video/keypoints/front")
detect_keypoints("Output Video/frames/side", "Output Video/keypoints/side")
detect_keypoints("Output Video/frames/top", "Output Video/keypoints/top")


#### Extract 2D Key Point Data into Json

In [ ]:
import cv2
import mediapipe as mp
import os
import json

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def extract_2d_keypoints(input_folder, output_json):
    """Extracts 2D keypoints from images and saves them to a JSON file."""
    keypoints_data = {}

    for img_name in sorted(os.listdir(input_folder)):  # Process frames in order
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue  # Skip if image is corrupted

        # Convert BGR to RGB (MediaPipe expects RGB)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_image)

        frame_keypoints = []
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                frame_keypoints.append((landmark.x, landmark.y))  # Store (x, y) only

        keypoints_data[img_name] = frame_keypoints  # Store keypoints for this frame

    # Save results to JSON
    with open(output_json, 'w') as f:
        json.dump(keypoints_data, f, indent=4)

    print(f"✅ 2D keypoints extracted and saved to {output_json}")

# Example usage (Extract for front, side, and top views)
extract_2d_keypoints("Output Video/frames/front", "Output Video/keypoints/front.json")
extract_2d_keypoints("Output Video/frames/side", "Output Video/keypoints/side.json")
extract_2d_keypoints("Output Video/frames/top", "Output Video/keypoints/top.json")


#### Normalize the data

- No need Because MediaPipe Already Normalize

#### Pose Completness Check

In [ ]:
import cv2
import mediapipe as mp
import os
import shutil

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def check_pose_completeness_and_move(input_folder, bad_folder, min_visible=25, visibility_threshold=0.5):
    """
    Checks pose completeness and moves frames with low keypoint visibility to a separate folder.
    """
    if not os.path.exists(bad_folder):
        os.makedirs(bad_folder)

    print(f"🔍 Checking pose completeness in: {input_folder}")
    incomplete_frames = []

    for img_name in sorted(os.listdir(input_folder)):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_image)

        visible_count = 0
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                if landmark.visibility >= visibility_threshold:
                    visible_count += 1

        if visible_count < min_visible:
            incomplete_frames.append((img_name, visible_count))
            print(f"⚠️ Incomplete pose in {img_name}: {visible_count}/33 visible")

            # Move the bad frame
            shutil.move(img_path, os.path.join(bad_folder, img_name))

    print(f"\n📉 Moved {len(incomplete_frames)} incomplete frames to '{bad_folder}'")
    return incomplete_frames


In [ ]:
# Move bad top-view frames
check_pose_completeness_and_move(
    input_folder="Output Video/frames/top",
    bad_folder="Output Video/frames/top_bad"
)

# You can run it for front or side too
# check_pose_completeness_and_move("Output Video/frames/front", "Output Video/frames/front_bad")


#### Visualize Annotations

In [ ]:
import matplotlib.pyplot as plt

# Load the image
image = plt.imread("Output Video/frames/front/frame_0000.jpg")
height, width, _ = image.shape

# Convert normalized keypoints to pixel values
x_pixels = [x * width for x, y in keypoints]
y_pixels = [y * height for x, y in keypoints]

# Display image and overlay points
plt.imshow(image)
plt.scatter(x_pixels, y_pixels, c="red", marker="o")  # Red dots for keypoints
plt.title("MediaPipe Pose Annotations")
plt.show()


#### Manually Update Annoatations

In [ ]:
import cv2
import numpy as np

image_path = "Output Video/frames/front/frame_0000.jpg"
image = cv2.imread(image_path)
height, width, _ = image.shape

# Replace with your actual keypoints (normalized)
keypoints = [
        [
            0.5505455732345581,
            0.5588361024856567
        ],
        [
            0.5572452545166016,
            0.552212119102478
        ],
        [
            0.5614649653434753,
            0.5519828200340271
        ],
        [
            0.5650237202644348,
            0.5518215894699097
        ],
        [
            0.5457234382629395,
            0.5524178743362427
        ],
        [
            0.542575478553772,
            0.5522381067276001
        ],
        [
            0.5397322773933411,
            0.5520192980766296
        ],
        [
            0.5709152817726135,
            0.551386833190918
        ],
        [
            0.5365363359451294,
            0.5507765412330627
        ],
        [
            0.5573853850364685,
            0.5633068680763245
        ],
        [
            0.5449458360671997,
            0.5633752346038818
        ],
        [
            0.6019920110702515,
            0.5733140110969543
        ],
        [
            0.5090041756629944,
            0.5722963213920593
        ],
        [
            0.6641108989715576,
            0.5980207920074463
        ],
        [
            0.4531230330467224,
            0.5994670391082764
        ],
        [
            0.6960875988006592,
            0.6336801648139954
        ],
        [
            0.4205382168292999,
            0.6375626921653748
        ],
        [
            0.7118327617645264,
            0.6432164907455444
        ],
        [
            0.410492867231369,
            0.6457881331443787
        ],
        [
            0.7016799449920654,
            0.6457731127738953
        ],
        [
            0.42393040657043457,
            0.6460126042366028
        ],
        [
            0.6945846676826477,
            0.6432654857635498
        ],
        [
            0.42746487259864807,
            0.6433942914009094
        ],
        [
            0.5882201790809631,
            0.6286242008209229
        ],
        [
            0.5343642830848694,
            0.6304098963737488
        ],
        [
            0.6483628749847412,
            0.6160215735435486
        ],
        [
            0.46625185012817383,
            0.6172221899032593
        ],
        [
            0.6089972853660583,
            0.656097948551178
        ],
        [
            0.5100529193878174,
            0.6553679704666138
        ],
        [
            0.6025288105010986,
            0.6603747606277466
        ],
        [
            0.518900454044342,
            0.658387303352356
        ],
        [
            0.6210933923721313,
            0.6730417609214783
        ],
        [
            0.47687360644340515,
            0.6730539202690125
        ]
    ]

# Convert to pixel coordinates
keypoints_px = [(int(x * width), int(y * height)) for x, y in keypoints]

# For dragging
dragging = False
drag_index = -1
undo_stack = []

def draw_points(img, points):
    for i, (x, y) in enumerate(points):
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(img, str(i), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)

def get_nearest_point(x, y, points, radius=10):
    for i, (px, py) in enumerate(points):
        if abs(px - x) <= radius and abs(py - y) <= radius:
            return i
    return -1

def mouse_callback(event, x, y, flags, param):
    global dragging, drag_index, keypoints_px, undo_stack

    if event == cv2.EVENT_LBUTTONDOWN:
        idx = get_nearest_point(x, y, keypoints_px)
        if idx != -1:
            dragging = True
            drag_index = idx
            undo_stack.append(keypoints_px.copy())  # Save previous state
    elif event == cv2.EVENT_MOUSEMOVE:
        if dragging and drag_index != -1:
            keypoints_px[drag_index] = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False
        drag_index = -1

cv2.namedWindow("Adjust Keypoints", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Adjust Keypoints", 640, 480)
cv2.setMouseCallback("Adjust Keypoints", mouse_callback)

while True:
    temp_image = image.copy()
    draw_points(temp_image, keypoints_px)
    cv2.imshow("Adjust Keypoints", temp_image)
    key = cv2.waitKey(1)

    if key == ord('q'):  # Quit and save
        break
    elif key == ord('u'):  # Undo
        if undo_stack:
            keypoints_px = undo_stack.pop()
            print("Undo last move.")

cv2.destroyAllWindows()

# Convert to normalized keypoints
updated_keypoints = [[x / width, y / height] for x, y in keypoints_px]
print("Updated normalized keypoints:")
for point in updated_keypoints:
    print(point)


#### Saving Image MetaData

In [ ]:
import os
import cv2
import json

def store_frame_metadata(image_folder, output_metadata_json):
    """
    Stores metadata (filename, resolution, etc.) for annotated frames in a JSON file.
    """
    metadata = {}

    for img_name in sorted(os.listdir(image_folder)):
        img_path = os.path.join(image_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        height, width, channels = image.shape
        metadata[img_name] = {
            "resolution": f"{width}x{height}",
            "width": width,
            "height": height,
            "channels": channels,
            "path": img_path
            # You can add more info here like timestamp if available
        }

    # Save metadata to JSON
    with open(output_metadata_json, 'w') as f:
        json.dump(metadata, f, indent=4)

    print(f"📝 Metadata saved to {output_metadata_json}")


In [ ]:
store_frame_metadata("Output Video/keypoints/front", "Output Video/keypoints/front_metadata.json")
store_frame_metadata("Output Video/keypoints/side", "Output Video/keypoints/side_metadata.json")
store_frame_metadata("Output Video/keypoints/top", "Output Video/keypoints/top_metadata.json")
